## Start

In [1]:
import clickhouse_connect
import datetime
import os
import pytz
import pandas as pd
from dateutil.relativedelta import relativedelta
from dotenv import load_dotenv

import sys
sys.path.append('/home/boris/Documents/Work/analytics/Clickhouse')
from clickhouse_client import ClickHouse_client
ch = ClickHouse_client()
pd.set_option('display.max_rows', 1000)

___
## Tags: #Tables

# Links:

[[citizens_st_mobile]]
[[subscriptions_st_mobile]]
[[entries_installation_points_dir_partner]]
[[installation_point_st_partner]]
[[citizen_payments_st_mobile]]
[[intercoms_st_partner]]

In [7]:
query_text = """--sql
    CREATE TABLE db1.t_subscribtions_citizens_by_companies_and_cities_address_ch 
    (
        `report_date` Date,
        `intercom_uuid` String,
        `motherboard_id` String,
        `city` String,
        `full_address` String,
        `company_name` String,
        `partner_uuid` String,
        `partner_lk` String,
        `tin` String,
        `citizen_id_in_flat_with_subscriptions` UInt32,
        `payments_amount` UInt32,
        `activated_citizen_id` UInt32,
        `subscribed_citizen_id` UInt32,
        `flat_uuid` UInt32,
        `address_uuid` String
    )
    ENGINE = MergeTree()
    ORDER BY report_date
    """
ch.query_run(query_text)


shape: (0, 0)
┌┐
╞╡
└┘

___

In [8]:
query_text = """--sql
CREATE MATERIALIZED VIEW db1.t_subscribtions_citizens_by_companies_and_cities_address_mv
REFRESH EVERY 1 DAY OFFSET 5 HOUR 27 MINUTE TO db1.t_subscribtions_citizens_by_companies_and_cities_address_ch AS 
WITH citizens_st_mobile AS(
	SELECT
		report_date,
		citizen_id,
		address_uuid,
		flat_uuid,
		state
	FROM db1.`citizens_st_mobile_ch` 
	WHERE `state` = 'activated'
		and report_date = yesterday()
	),
	subscriptions_st_mobile AS(
	SELECT 
		report_date,
		citizen_id,
		state
	FROM db1.subscriptions_st_mobile_ch
	WHERE state = 'activated'
		and report_date = yesterday()
	),
	installation_point_st AS (
	SELECT 
		report_date,
		installation_point_id
	FROM db1.installation_point_st_partner_ch
	WHERE report_date= yesterday()
	),
	intercoms_st_partner AS (
	SELECT
		report_date,
		intercom_uuid,
		partner_uuid,
		installation_point_id
	FROM db1.intercoms_st_partner_ch 
	WHERE report_date = yesterday()
	),
	citizen_payments_st_mobile AS (
	SELECT
		report_date,
		citizen_id,
		amount
	FROM db1.citizen_payments_st_mobile_ch 
	WHERE state = 'success'
		AND report_date = yesterday()
	),
	t4 AS(
	SELECT  
		intercoms_st_partner.report_date AS report_date,
		city,
		company_name,
		partner_lk,
		intercoms_st_partner.intercom_uuid AS intercom_uuid,
		intercoms_st_partner.partner_uuid AS partner_uuid,
		motherboard_id,
		full_address,
		address_uuid,
		tin
	FROM intercoms_st_partner
	LEFT JOIN  db1.intercoms_dir_asgard_ch AS intercoms_dir_asgard
			ON intercoms_dir_asgard.intercom_uuid = intercoms_st_partner.intercom_uuid
	LEFT JOIN db1.companies_dir_partner_ch AS companies_dir_partner 
			ON companies_dir_partner.partner_uuid = intercoms_st_partner.partner_uuid
	LEFT JOIN installation_point_st
		ON installation_point_st.`installation_point_id` = intercoms_st_partner.`installation_point_id`
		AND installation_point_st.`report_date` = intercoms_st_partner.`report_date`
	LEFT JOIN db1.entries_installation_points_dir_partner_ch AS entries_installation_points 
			ON installation_point_st.`installation_point_id` = entries_installation_points.`installation_point_id`
	),
	--
	t1 AS (SELECT
		citizens_st_mobile.report_date AS report_date,
		intercoms_st_partner.intercom_uuid AS intercom_uuid,
		COUNT(DISTINCT if(citizens_st_mobile.citizen_id !=0, citizens_st_mobile.citizen_id ,Null)) as activated_citizen_id,	
		COUNT(DISTINCT if(subscriptions_st_mobile.citizen_id !=0, subscriptions_st_mobile.citizen_id ,Null)) as subscribed_citizen_id,	
		COUNT(DISTINCT if(citizens_st_mobile.flat_uuid !='', citizens_st_mobile.flat_uuid ,Null)) as flat_uuid
	FROM citizens_st_mobile
	LEFT JOIN  subscriptions_st_mobile
			ON citizens_st_mobile.`citizen_id` = subscriptions_st_mobile.`citizen_id`
			AND citizens_st_mobile.`report_date` = subscriptions_st_mobile.`report_date`
	LEFT JOIN db1.`entries_installation_points_dir_partner_ch` AS entries_installation_points 
			ON citizens_st_mobile.`address_uuid` = entries_installation_points.`address_uuid`
	LEFT JOIN installation_point_st
			ON entries_installation_points.`installation_point_id` = installation_point_st.`installation_point_id`
			AND installation_point_st.`report_date` = citizens_st_mobile.`report_date`
	LEFT JOIN intercoms_st_partner 
			ON intercoms_st_partner.installation_point_id = installation_point_st.installation_point_id 
			AND intercoms_st_partner.report_date = citizens_st_mobile.report_date 
	LEFT JOIN  db1.intercoms_dir_asgard_ch AS intercoms_dir_asgard
			ON intercoms_dir_asgard.intercom_uuid = intercoms_st_partner.intercom_uuid
	LEFT JOIN db1.companies_dir_partner_ch AS companies_dir_partner 
			ON companies_dir_partner.partner_uuid = intercoms_st_partner.partner_uuid
	GROUP BY report_date,
			intercom_uuid
	),
	--
	t2 AS (SELECT
		citizen_payments_st_mobile.report_date AS report_date,
		intercoms_st_partner.intercom_uuid AS intercom_uuid,
		sum(amount) AS payments_amount
	FROM citizen_payments_st_mobile
	LEFT JOIN citizens_st_mobile 
			ON citizen_payments_st_mobile.citizen_id = citizens_st_mobile.citizen_id  
			AND citizen_payments_st_mobile.report_date = citizens_st_mobile.report_date 
	LEFT JOIN db1.`entries_installation_points_dir_partner_ch` AS entries_installation_points 
			ON citizens_st_mobile.`address_uuid` = entries_installation_points.`address_uuid`
	LEFT JOIN installation_point_st
			ON entries_installation_points.`installation_point_id` = installation_point_st.`installation_point_id` 
			AND installation_point_st.`report_date` = citizen_payments_st_mobile.`report_date`
	LEFT JOIN intercoms_st_partner 
			ON intercoms_st_partner.installation_point_id = installation_point_st.installation_point_id 
			AND intercoms_st_partner.report_date = citizen_payments_st_mobile.report_date
	LEFT JOIN db1.companies_dir_partner_ch AS companies_dir_partner 
			ON companies_dir_partner.partner_uuid = intercoms_st_partner.partner_uuid
	LEFT JOIN  db1.intercoms_dir_asgard_ch AS intercoms_dir_asgard
			ON intercoms_dir_asgard.intercom_uuid = intercoms_st_partner.intercom_uuid
	GROUP BY 	
			report_date,
			intercom_uuid
	),
	--
	t3 AS (SELECT
		report_date,
		intercom_uuid,
		count(if(citizen_id !=0,citizen_id,NULL)) AS citizen_id_in_flat_with_subscriptions
	FROM
		(SELECT
			installation_point_st.report_date AS report_date,
			installation_point_st.citizen_id AS citizen_id,
			installation_point_st.installation_point_id AS installation_point_id,
			installation_point_st.flat_with_sub_active AS flat_with_sub_active,
			intercom_uuid
		FROM
			(SELECT 
				citizens_st_mobile.report_date AS report_date,
				citizens_st_mobile.citizen_id AS citizen_id,
				installation_point_st.installation_point_id AS installation_point_id,
				if(subscriptions_st_mobile.state = 'activated',1,0) AS if_sub_active,
				max(if_sub_active) OVER (partition by citizens_st_mobile.flat_uuid, citizens_st_mobile.report_date ORDER BY citizens_st_mobile.report_date DESC) AS flat_with_sub_active
			FROM citizens_st_mobile AS citizens_st_mobile
			LEFT JOIN subscriptions_st_mobile 
				ON citizens_st_mobile.`citizen_id` = subscriptions_st_mobile.`citizen_id`
				AND citizens_st_mobile.`report_date` = subscriptions_st_mobile.`report_date`
			LEFT JOIN db1.`entries_installation_points_dir_partner_ch` AS entries_installation_points 
				ON citizens_st_mobile.`address_uuid` = entries_installation_points.`address_uuid`
			LEFT JOIN db1.`installation_point_st_partner_ch` AS  installation_point_st
				ON entries_installation_points.`installation_point_id` = installation_point_st.`installation_point_id`
				AND installation_point_st.`report_date` = citizens_st_mobile.`report_date`) AS installation_point_st
			LEFT JOIN intercoms_st_partner 
				ON intercoms_st_partner.installation_point_id = installation_point_st.installation_point_id 
				AND intercoms_st_partner.report_date = installation_point_st.report_date
		WHERE flat_with_sub_active = 1
		)
	GROUP BY report_date,
			intercom_uuid
	)
	--
	SELECT
		t1.report_date AS report_date,
		t1.intercom_uuid AS intercom_uuid,
		t4.full_address AS full_address,
		t4.company_name AS company_name,
		t4.city AS city,
		t4.partner_uuid AS partner_uuid,
		t4.partner_lk AS partner_lk,
		t4.motherboard_id AS motherboard_id,
		t4.tin AS tin,
		t4.address_uuid AS address_uuid,
		payments_amount,
		activated_citizen_id,
		subscribed_citizen_id,
		flat_uuid,
		citizen_id_in_flat_with_subscriptions
	FROM  t1 
	LEFT JOIN t2 ON t1.report_date = t2.report_date
				AND t1.intercom_uuid = t2.intercom_uuid
	LEFT JOIN t3 ON t1.report_date = t3.report_date
				AND t1.intercom_uuid = t3.intercom_uuid
	LEFT JOIN t4  ON t1.report_date = t4.report_date
				AND t1.intercom_uuid = t4.intercom_uuid
--	SETTINGS join_algorithm = 'partial_merge'
	"""
ch.query_run(query_text)

shape: (0, 0)
┌┐
╞╡
└┘

In [9]:
query_text = """
    SELECT
        *
    FROM db1.t_subscribtions_citizens_by_companies_and_cities_address_ch
    WHERE intercom_uuid !=''
        AND full_address !=''
    ORDER BY report_date DESC
    LIMIT 10

    """
ch.query_run(query_text)

report_date,intercom_uuid,motherboard_id,city,full_address,company_name,partner_uuid,partner_lk,tin,citizen_id_in_flat_with_subscriptions,payments_amount,activated_citizen_id,subscribed_citizen_id,flat_uuid,address_uuid
date,str,str,str,str,str,str,str,str,i64,i64,i64,i64,i64,str
2025-08-19,"""898b7502-0cf3-490e-be80-b712fa…","""2018115147""","""Нефтекамск""","""Россия, Республика Башкортоста…","""Новотелеком ООО""","""7775bc0f-24e5-4afc-9054-e194c8…","""120174""","""5406260827""",0,0,39,0,19,"""fe821960-9255-4bbe-8535-804cf9…"
2025-08-19,"""78fb22f4-846e-4744-a076-66c6b7…","""2121125140""","""Тольятти""","""Россия, Самарская область, гор…","""Цифрал Тольятти""","""e7e86c75-fc8b-4710-839d-56ad86…","""152762""","""""",0,0,43,0,20,"""a67afaca-07d4-472c-a0f0-5bd448…"
2025-08-19,"""5c9ccb53-ebe8-484b-ace0-694563…","""2121123801""","""Мурманск""","""Россия, Мурманская область, го…","""ООО Домашний мастер ПРО""","""15a6bb4b-1cc7-4686-8cda-dada01…","""123999""","""5190145561""",10,0,25,3,13,"""28a51ee0-1979-4636-8076-5b63cf…"
2025-08-19,"""fe40087e-a567-4698-abee-30dd18…","""2422252756""","""Саки""","""Россия, Республика Крым, город…","""ИП Сумулиди Д.П. ПРО""","""5345bacf-5a93-4d61-b626-ac86ab…","""144666""","""910200086888""",7,69,16,2,7,"""cd26751e-4b70-4649-9d70-2eda6a…"
2025-08-19,"""6ab880a0-78f5-4ffe-89d3-9cc060…","""2236149597""","""Новокузнецк""","""Россия, Кемеровская область — …","""ООО Комфорт-Сервис видео ПРО""","""c5daca2a-5cd7-4bd1-9072-cd1a4c…","""121764""","""4253005871""",7,1,24,4,11,"""95ad7402-9b4c-4f0a-b3b5-c0c1f8…"
2025-08-19,"""1bce72f1-cd1a-4054-9d9e-5cb977…","""2101115884""","""Киров""","""Россия, Кировская область, мун…","""ООО Добрыня ПРО""","""1c64b848-e82c-46ee-9d0d-c74093…","""121833""","""4345434743""",6,0,29,2,15,"""d9ae2759-0779-40ad-9c9d-0e37a1…"
2025-08-19,"""d4b8e8b4-c2e9-49c8-8f7d-5c71cd…","""2235145519""","""Краснодар""","""Россия, Краснодарский край, му…","""ГК Точно - Умный Дом""","""fbe76e25-3efd-4448-91b7-5953c7…","""155178""","""2310221660""",9,0,65,6,43,"""3bb2051f-97f2-457d-ae07-be406c…"
2025-08-19,"""bd696475-5ab2-4047-a205-8cea62…","""2329250163""","""Горячий Ключ""","""Россия, Краснодарский край, го…","""ДОМИНАНТ КРД""","""684986ca-87e7-4d53-8a3c-8cb47a…","""125759""","""2312305918""",1,0,7,1,5,"""ede59c5c-a472-495a-a261-8ad167…"
2025-08-19,"""67042cc9-2db7-419d-838e-ee30fb…","""2125126723""","""Санкт-Петербург""","""Россия, Санкт-Петербург, Санкт…","""ООО АВЕНТИН СЕРВИС ПРО""","""f58ba743-24cc-4ccb-8832-c37343…","""120863""","""7801577995""",0,0,15,0,10,"""257bcc45-0102-4019-9425-f4b41e…"


In [ ]:
query_text = """
SYSTEM REFRESH VIEW db1.t_subscribtions_citizens_by_companies_and_cities_address_mv
"""

ch.query_run(query_text)

,elapsed_ns,query_id
0,1313005,942ed803-976d-4ff9-ad7e-e1a5a1b371f7


In [5]:
query_text = """
    DROP TABLE db1.t_subscribtions_citizens_by_companies_and_cities_address_ch
    """
ch.query_run(query_text)

elapsed_ns,query_id
i64,str
6906711,"""0aca6e68-0ec7-45d1-be87-99701f…"


In [6]:
query_text = """
    DROP TABLE db1.t_subscribtions_citizens_by_companies_and_cities_address_mv
    """
ch.query_run(query_text)

elapsed_ns,query_id
i64,str
1327562,"""4cf8c0a3-137e-48e3-b593-744444…"
